# Príprava dát

### Poznámka: notebook bol spustený na verzii Python 3.10. Všetky cesty v notebooku sú relatívne k priečinku, v ktorom sa nachádzate.

### Import,stiahnutie všetkých potrebných knižníc a rozbalenie archívu

!pip install pylabel

In [1]:
import os
import shutil
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import IPython
import json
import os
import json
import csv
import time
from pathlib import Path
from pylabel import importer

In [10]:
!unzip Meteory.zip

Archive:  Meteory.zip
   creating: Meteory/images/
  inflating: Meteory/images/20160101_183456_VAZEC.jpg  
  inflating: Meteory/images/20160101_203345_VAZEC.jpg  
  inflating: Meteory/images/20160101_215242_VAZEC.jpg  
  inflating: Meteory/images/20160101_220714_VAZEC.jpg  
  inflating: Meteory/images/20160102_001718_VAZEC.jpg  
  inflating: Meteory/images/20160102_015154_VAZEC.jpg  
  inflating: Meteory/images/20160102_015604_VAZEC.jpg  
  inflating: Meteory/images/20160102_015754_VAZEC.jpg  
  inflating: Meteory/images/20160102_022532_VAZEC.jpg  
  inflating: Meteory/images/20160102_023324_VAZEC.jpg  
  inflating: Meteory/images/20160102_025416_VAZEC.jpg  
  inflating: Meteory/images/20160102_031609_VAZEC.jpg  
  inflating: Meteory/images/20160102_031830_VAZEC.jpg  
  inflating: Meteory/images/20160102_032357_VAZEC.jpg  
  inflating: Meteory/images/20160102_044433_VAZEC.jpg  
  inflating: Meteory/images/20160102_222546_VAZEC.jpg  
  inflating: Meteory/images/20160102_224201_VAZEC.jpg

In [8]:
shutil.rmtree('Meteory_format_YOLOv5_aug')

### Rozdelenie údajov na trénovaciu, validačnú a testovaciu množinu

In [12]:
base_path = 'Meteory'
folders = ['train', 'val', 'test']

for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

for folder in folders:
    file_path = os.path.join(base_path, f"{folder}.txt")
    
    with open(file_path, 'r') as file:
        for line in file:
            image_path = line.strip()
            
            if image_path.startswith('./'):
                image_path = image_path[2:]
                
            full_image_path = os.path.join(base_path, image_path)
            dest_path = os.path.join(base_path, folder, os.path.basename(image_path))
            
            if os.path.exists(full_image_path):
                shutil.move(full_image_path, dest_path)
                print(f"Photo {full_image_path} moved to {dest_path}")
            else:
                print(f"File not found: {full_image_path}")


File not found: Meteory/images/20160101_183456_VAZEC.jpg
File not found: Meteory/images/20160101_203345_VAZEC.jpg
File not found: Meteory/images/20160101_215242_VAZEC.jpg
File not found: Meteory/images/20160101_220714_VAZEC.jpg
File not found: Meteory/images/20160102_001718_VAZEC.jpg
File not found: Meteory/images/20160102_015154_VAZEC.jpg
File not found: Meteory/images/20160102_015604_VAZEC.jpg
File not found: Meteory/images/20160102_015754_VAZEC.jpg
File not found: Meteory/images/20160102_022532_VAZEC.jpg
File not found: Meteory/images/20160102_023324_VAZEC.jpg
File not found: Meteory/images/20160102_031609_VAZEC.jpg
File not found: Meteory/images/20160102_031830_VAZEC.jpg
File not found: Meteory/images/20160102_032357_VAZEC.jpg
File not found: Meteory/images/20160102_044433_VAZEC.jpg
File not found: Meteory/images/20160102_222546_VAZEC.jpg
File not found: Meteory/images/20160103_210301_VAZEC.jpg
File not found: Meteory/images/20160103_212049_VAZEC.jpg
File not found: Meteory/images/

### Použitie horizotálneho prevrátenia 

In [13]:
#horizontalne prevratenie
base_path = 'Meteory/train'

for filename in os.listdir(base_path):
    if filename.endswith(".jpg"): 
        original_path = os.path.join(base_path, filename)

        with Image.open(original_path) as img:
            flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)

            flipped_path = os.path.join(base_path, f"flipped_{filename}")

            flipped_img.save(flipped_path)

            print(f"Rotated photo saved: {flipped_path}")


/tmp/ipykernel_110/3794462017.py:9: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)


Rotated photo saved: Meteory/train/flipped_20160101_183456_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160101_203345_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160101_215242_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160101_220714_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_001718_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_015154_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_015604_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_015754_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_022532_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_023324_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_031609_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_031830_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_032357_VAZEC.jpg
Rotated photo saved: Meteory/train/flipped_20160102_044433_VAZEC.jpg
Rotated photo saved: Meteory/train

### Vytváranie anotácií pre prevrátené obrázky

In [14]:
#vytvorenie labelov pre prevratene obrazky

labels_path = 'Meteory/labels'
images_path = 'Meteory/train'

flipped_images = {file.replace('flipped_', '').replace('.jpg', '') for file in os.listdir(images_path) if 'flipped_' in file}

for filename in os.listdir(labels_path):
    if filename.replace('.txt', '') in flipped_images:
        with open(os.path.join(labels_path, filename), 'r') as file:
            lines = file.readlines()


        with open(os.path.join(labels_path, f"flipped_{filename}"), 'w') as new_file:

            for line in lines:
                parts = line.split()
                if len(parts) >= 5:

                    x = float(parts[1])
                    new_x = 1 - x

                    new_line = f"{parts[0]} {new_x:.5f} {parts[2]} {parts[3]} {parts[4]}\n"
                    new_file.write(new_line)
            
            print(f"Changed annot saved in: {new_file.name}")


Changed annot saved in: Meteory/labels/flipped_20160101_183456_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160101_203345_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160101_215242_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160101_220714_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_001718_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_015154_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_015604_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_015754_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_022532_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_023324_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_031609_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_031830_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_20160102_032357_VAZEC.txt
Changed annot saved in: Meteory/labels/flipped_2016

### Vytvorenie súboru údajov, ktorý sa používa pri tréningu HIC-YOLOv5 a YOLO-NAS

In [15]:
base_source = 'Meteory'
base_dataset = 'Meteory_format_YOLOv5_aug'
folders = ['train', 'test', 'val']


for folder in folders:
    images_path = os.path.join(base_dataset, folder, 'images')
    labels_path = os.path.join(base_dataset, folder, 'labels')
    os.makedirs(images_path, exist_ok=True)
    os.makedirs(labels_path, exist_ok=True)


for folder in folders:
    images_source = os.path.join(base_source, folder)
    labels_source = os.path.join(base_source, 'labels')
    images_dest = os.path.join(base_dataset, folder, 'images')
    labels_dest = os.path.join(base_dataset, folder, 'labels')
    

    for image_file in os.listdir(images_source):
        source_image_path = os.path.join(images_source, image_file)
        dest_image_path = os.path.join(images_dest, image_file)
        
        if os.path.isfile(source_image_path):
            shutil.copy2(source_image_path, dest_image_path)
            

            label_file = os.path.splitext(image_file)[0] + '.txt'
            source_label_path = os.path.join(labels_source, label_file)
            dest_label_path = os.path.join(labels_dest, label_file)
            
            if os.path.isfile(source_label_path):
                shutil.copy2(source_label_path, dest_label_path)
                print(f"Copied {source_image_path} and {source_label_path} to {images_dest} and {labels_dest}")

print("All files have been copied successfully.")


Copied Meteory/train/20160101_183456_VAZEC.jpg and Meteory/labels/20160101_183456_VAZEC.txt to Meteory_format_YOLOv5_aug/train/images and Meteory_format_YOLOv5_aug/train/labels
Copied Meteory/train/20160101_203345_VAZEC.jpg and Meteory/labels/20160101_203345_VAZEC.txt to Meteory_format_YOLOv5_aug/train/images and Meteory_format_YOLOv5_aug/train/labels
Copied Meteory/train/20160101_215242_VAZEC.jpg and Meteory/labels/20160101_215242_VAZEC.txt to Meteory_format_YOLOv5_aug/train/images and Meteory_format_YOLOv5_aug/train/labels
Copied Meteory/train/20160101_220714_VAZEC.jpg and Meteory/labels/20160101_220714_VAZEC.txt to Meteory_format_YOLOv5_aug/train/images and Meteory_format_YOLOv5_aug/train/labels
Copied Meteory/train/20160102_001718_VAZEC.jpg and Meteory/labels/20160102_001718_VAZEC.txt to Meteory_format_YOLOv5_aug/train/images and Meteory_format_YOLOv5_aug/train/labels
Copied Meteory/train/20160102_015154_VAZEC.jpg and Meteory/labels/20160102_015154_VAZEC.txt to Meteory_format_YOLOv

### !!!V tejto fáze musíte vytvoriť súbor data.yaml v priečinku Meteory_format_YOLOv5_aug, aby YOLO fungovalo správne, vyzerá takto:

## Vytváranie anotácií COCO

In [16]:
!mkdir Meteory_format_COCO_aug
!mkdir Meteory_format_COCO_aug/train
!mkdir Meteory_format_COCO_aug/test
!mkdir Meteory_format_COCO_aug/valid

### Vytvorenie samotných súborov JSON s anotáciami obrázkov, ktoré sú anotáciami COCO

In [57]:
path_to_annotations = "Meteory_format_YOLOv5/train/labels"
path_to_images = "Meteory_format_YOLOv5/train/images"
yoloclasses = ['meteor', 'satelite']
dataset = importer.ImportYoloV5(path=path_to_annotations, path_to_images=path_to_images, cat_names=yoloclasses,
    img_ext="jpg", name="coco128")
dataset.export.ExportToCoco(cat_id_index=1)







Importing YOLO files...:   0%|          | 0/8432 [00:00<?, ?it/s]

AssertionError: No image file found: temp/train/labels/temp/train/images/20160101_183456_VAZEC.jpg. Check path_to_images and img_ext arguments.

In [50]:
path_to_annotations = "Meteory_format_YOLOv5/val/labels"
path_to_images = "Meteory_format_YOLOv5/val/images"
yoloclasses = ['meteor', 'satelite']
dataset = importer.ImportYoloV5(path=path_to_annotations, path_to_images=path_to_images, cat_names=yoloclasses,
    img_ext="jpg", name="coco128")
dataset.export.ExportToCoco(cat_id_index=1)

Meteory			 Meteory_format_YOLOv5_aug  preprocessing.ipynb
Meteory_format_COCO_aug  Meteory.zip		    temp


In [ ]:
path_to_annotations = "Meteory_format_YOLOv5/test/labels"
path_to_images = "Meteory_format_YOLOv5/test/images"
yoloclasses = ['meteor', 'satelite']
dataset = importer.ImportYoloV5(path=path_to_annotations, path_to_images=path_to_images, cat_names=yoloclasses,
    img_ext="jpg", name="coco128")
dataset.export.ExportToCoco(cat_id_index=1)

In [59]:
!cp dataset/test/labels/coco128.json Meteory_format_COCO_aug/test
!cp dataset/train/labels/coco128.json Meteory_format_COCO_aug/train
!cp dataset/valid/labels/coco128.json Meteory_format_COCO_aug/valid

cp: cannot stat 'temp/test/labels/coco128.json': No such file or directory
cp: cannot stat 'temp/train/labels/coco128.json': No such file or directory
cp: cannot stat 'temp/valid/labels/coco128.json': No such file or directory


In [ ]:
old_name_train = "Meteory_format_COCO_aug/train/coco128.json"
new_name_train = "Meteory_format_COCO_aug/train/_annotations.coco"

old_name_valid = "Meteory_format_COCO_aug/valid/coco128.json"
new_name_valid = "Meteory_format_COCO_aug/valid/_annotations.coco"

old_name_test = "Meteory_format_COCO_aug/test/coco128.json"
new_name_test = "Meteory_format_COCO_aug/test/_annotations.coco"

os.rename(old_name_train, new_name_train)
os.rename(old_name_valid, new_name_valid)
os.rename(old_name_test, new_name_test)

In [20]:
source_dirs = {
    "train": "Meteory_format_YOLOv5_aug/train/images",
    "valid": "Meteory_format_YOLOv5_aug/val/images",
    "test": "Meteory_format_YOLOv5_aug/test/images"
}

target_dirs = {
    "train": "Meteory_format_COCO_aug/train",
    "valid": "Meteory_format_COCO_aug/valid",
    "test": "Meteory_format_COCO_aug/test"
}


def copy_files(source_dir, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    for filename in os.listdir(source_dir):
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)
        
        if os.path.isfile(source_file):
            shutil.copy2(source_file, target_file)

for key in source_dirs:
    copy_files(source_dirs[key], target_dirs[key])
print("Copied")


Copied


In [24]:
import shutil

# Specify the folder path you want to zip
folder_path = 'Meteory_format_YOLOv5_aug'

# Specify the name for the zip file
zip_filename = 'Meteory_format_YOLOv5_aug'

# Create a zip file
shutil.make_archive(zip_filename, 'zip', folder_path)

'/home/jovyan/work/test2/Meteory_format_YOLOv5_aug.zip'